In [1]:
import os

os.chdir('/ictstr01/home/icb/fatemehs.hashemig/codes/interpretable-ssl')

In [2]:
import importlib

In [3]:
from experiment_evaluator import *

INFO:pytorch_lightning.utilities.seed:Global seed set to 0
/home/icb/fatemehs.hashemig/miniconda3/envs/apex-env/lib/python3.12/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/home/icb/fatemehs.hashemig/miniconda3/envs/apex-env/lib/python3.12/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)
 captum (see https://github.com/pytorch/captum).


In [4]:
from interpretable_ssl.datasets.hlca import *

In [5]:
ds = HLCADataset()

ee = ExperimentEvaluator(ds)

loading hlca data


INFO:root:Starting '__init__' of class 'get train test'


done


INFO:root:Finished '__init__' of class 'get train test' in 3.4247 seconds


In [6]:
td = {'swav': {
            "dimensionality_reduction": ["pca"],
            "prot_decoding_loss_scaler": [0],
            "cvae_loss_scaler": [0.0],
            "k_neighbors": [7],
            "training_type": ['semi_supervised'],
            "fine_tuning_epochs": [150],
            "pretraining_epochs": [150],
            "augmentation_type": ["knn"],
            'batch_size': [1024],
            'dataset_id':['hlca'],
            'model_name_version': [3.5],
            "nmb_crops": [[3]],
    
        }}

trainers = ee.generate_trainers(td)

Generating trainers for swav: 100%|██████████| 1/1 [00:00<00:00, 27.85it/s, Memory=14.0%, CPU=0.7%, trainer_name=swav_aug_knn_DR_pca_kN_7_ds_hlca]


In [25]:
import interpretable_ssl.evaluation.metrics
importlib.reload(interpretable_ssl.evaluation.metrics)

import interpretable_ssl.evaluation.metric_generator
importlib.reload(interpretable_ssl.evaluation.metric_generator)
from interpretable_ssl.evaluation.metric_generator import *

In [26]:
mg = MetricGenerator(trainers[0])

calc = mg.get_query_calculator()

Embedding dictionary:
 	Num conditions: [9]
 	Embedding dim: [10]
Encoder Architecture:
	Input Layer in, out and cond: 2000 45 10
	Mean/Var Layer in/out: 45 8
Decoder Architecture:
	First Layer in, out and cond:  8 45 10
	Output Layer in/out:  45 2000 

Embedding dictionary:
 	Num conditions: [11]
 	Embedding dim: [10]
Encoder Architecture:
	Input Layer in, out and cond: 2000 45 10
	Mean/Var Layer in/out: 45 8
Decoder Architecture:
	First Layer in, out and cond:  8 45 10
	Output Layer in/out:  45 2000 



100%|██████████| 36/36 [00:03<00:00, 11.18it/s]


In [15]:
mg.load_metrics()

# debug knn results

In [40]:
import interpretable_ssl.evaluation.knn
importlib.reload(interpretable_ssl.evaluation.knn)
from interpretable_ssl.evaluation.knn import *

In [41]:
embeddings, labels = calc.latents[0], self.adata.obs[self.label_key]
test_size=0.2
n_neighbors=5
X_train, X_test, y_train, y_test, label_encoder = prepare_data(embeddings, labels, test_size)


In [42]:
len(trainers[0].query), len(X_train) + len(X_test)

(36412, 36411)

In [43]:
knn = train_knn_classifier(X_train, y_train, n_neighbors)

In [44]:
y_pred = knn.predict(X_test)

In [45]:
# Calculate F1 scores for each class and for micro, macro, and weighted averages
class_f1_scores = f1_score(y_test, y_pred, average=None)
micro_f1 = f1_score(y_test, y_pred, average='micro')
macro_f1 = f1_score(y_test, y_pred, average='macro')
weighted_f1 = f1_score(y_test, y_pred, average='weighted')

In [48]:
# Prepare a DataFrame for output
results = {
    'Class': label_encoder.classes_ if label_encoder else np.unique(y_test),
    'F1 Score': class_f1_scores
}

IndentationError: unexpected indent (1812737019.py, line 2)

In [47]:
len(label_encoder.classes_ )

43

# debugging calc

In [54]:
self = calc
scores = knn_classifier_with_f1_report(calc.latents[0], self.adata.obs[self.label_key])

In [56]:
scores

,Class,F1 Score
0,B cell,0.777778
1,CD1c-positive myeloid dendritic cell,0.642157
2,"CD4-positive, alpha-beta T cell",0.876014
3,"CD8-positive, alpha-beta T cell",0.828685
4,T cell,0.000000
5,alveolar macrophage,0.918935
6,alveolar type 1 fibroblast cell,0.737430
7,alveolar type 2 fibroblast cell,0.823129
8,bronchial goblet cell,0.000000
9,bronchus fibroblast of lung,0.333333


In [60]:
calc.knn_results()

,macro,micro,weighted
model,,,
swav_aug_knn_DR_pca_kN_7_ds_hlca,0.834821,0.525669,0.830443


# debug linear

In [21]:
linear = calc.linear_results(10)

In [22]:
knn = calc.knn_results()

In [27]:
pd.concat([linear, knn, None], axis=1)

,linear classifier f1 macro,linear classifier f1 micro,linear classifier f1 weighted,knn f1 macro,knn f1 micro,knn f1 weighted
model,,,,,,
swav_aug_knn_DR_pca_kN_7_ds_hlca,0.728134,0.238236,0.673691,0.851023,0.511129,0.846343


In [28]:
metrics = mg.generate_metrics()

Embedding dictionary:
 	Num conditions: [9]
 	Embedding dim: [10]
Encoder Architecture:
	Input Layer in, out and cond: 2000 45 10
	Mean/Var Layer in/out: 45 8
Decoder Architecture:
	First Layer in, out and cond:  8 45 10
	Output Layer in/out:  45 2000 

Embedding dictionary:
 	Num conditions: [11]
 	Embedding dim: [10]
Encoder Architecture:
	Input Layer in, out and cond: 2000 45 10
	Mean/Var Layer in/out: 45 8
Decoder Architecture:
	First Layer in, out and cond:  8 45 10
	Output Layer in/out:  45 2000 



100%|██████████| 36/36 [00:03<00:00, 11.86it/s]


Skipped cell type plasmacytoid dendritic cell, due to < 10 cells
Skipped cell type multi-ciliated epithelial cell, due to < 10 cells
Skipped cell type tracheobronchial goblet cell, due to < 10 cells
Skipped cell type lung neuroendocrine cell, due to < 10 cells
Skipped cell type hematopoietic stem cell, due to < 10 cells
Skipped cell type ionocyte, due to < 10 cells
Processing batches, calcualte centroids and pairwise distances


  0%|          | 0/2 [00:00<?, ?it/s]

Metrics:   0%|          | 0/10 [00:00<?, ?it/s, Bio conservation: isolated_labels]INFO:root:isolated labels: no more than 1 batches per label
INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': 
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory

Metrics:  60%|██████    | 6/10 [00:43<00:20,  5.03s/it, Batch correction: kbet_per_label]

INFO     hematopoietic stem cell consists of a single batch or is too small. Skip.                                 
INFO     ionocyte consists of a single batch or is too small. Skip.                                                
INFO     lung neuroendocrine cell consists of a single batch or is too small. Skip.                                
INFO     multi-ciliated epithelial cell consists of a single batch or is too small. Skip.                          
INFO     plasmacytoid dendritic cell consists of a single batch or is too small. Skip.                             
INFO     smooth muscle cell consists of a single batch or is too small. Skip.                                      
INFO     tracheobronchial goblet cell consists of a single batch or is too small. Skip.                            



Embeddings: 100%|██████████| 1/1 [01:13<00:00, 73.85s/it]atch correction: pcr_comparison]

                                                                                         

TypeError: Index(...) must be called with a collection of some kind, 'swav_aug_knn_DR_pca_kN_7_ds_hlca' was passed